In [2]:
# QUESTION 1

#pip install beautifulsoup4
pip install "ipython-beautifulsoup[bs4]"

     |████████████████████████████████| 122kB 17.4MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install lxml

     |████████████████████████████████| 5.5MB 6.5MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'html5lib')
table = soup.table
import pandas as pd
df = pd.read_html(str(table))[0]
df = df[df["Borough"] != "Not assigned"].reset_index(drop=True)
df.loc[df["Neighbourhood"] == "Not assigned", "Neighbourhood"] = df["Borough"]
df_grouped = df.groupby(["Postal Code","Borough"])["Neighbourhood"].apply(','.join).reset_index()
df_final = df_grouped
shape = df_final.shape
text = "This is the dataframe's shape: {}".format(shape)
print(text)
df_final.head()

This is the dataframe's shape: (103, 3)


,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [4]:
# QUESTION 2

loc_data = pd.read_csv("https://cocl.us/Geospatial_data")
df_merge = pd.merge(loc_data, df_final,  how='outer')
shape2 = df_merge.shape
text2 = "This is the dataframe's shape: {}".format(shape2)
print(text2)
df_merge.head()

This is the dataframe's shape: (103, 5)


,Postal Code,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [5]:
# QUESTION 3 

!conda install -c conda-forge folium=0.5.0 --yes
import folium
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    pandas-1.1.5               |   py36h284efc9_0        11.3 MB  conda-forge
    toolz-0.11.1               |             py_0          46 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:        12.1 MB

The following NEW p

In [6]:
address = 'Toronto, CA'
geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of toronto are {}, {}.'.format(latitude, longitude))
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

The geograpical coordinate of toronto are 43.6534817, -79.3839347.


In [7]:
# Let's see the gerographical coordinates on the map.

for lat, lng, borough, neighborhood in zip(df_merge['Latitude'], df_merge['Longitude'], df_merge['Borough'],df_merge['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
map_toronto

In [21]:
# The borough of Scarborough has been at the top of my dataframes all this time, which has made me quite curious about this borough. 
# Let's zoom in on it and see what's popular in the neighbourhoods of this rough-sounding borough. 
# Due to there being multiple neighbourhoods per postal code, I'll be using the postal code to keep things easier to read.



In [24]:
#First up: importing the needed libraries
import requests 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!pip install folium
import folium
print("Done importing")

Done importing


In [8]:
# hidden credentials
CLIENT_ID = 'VX3QIBY0VVYW1RLPVI2YBJPEWITP11HQNARLMFSGNIRN1DFE' # your Foursquare ID
CLIENT_SECRET = '3A4YZ0YMW4IBLVSBEIZLX2DVUKQRPDPB1PV1ETYJDEJ0LPMU'

In [9]:
# Other required fields to make API calls to Foursquare
VERSION = '20180605' 
LIMIT = 100

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
# Let's zoom in on Scarborough
scar_data = df_merge[df_merge["Borough"] == "Scarborough"].reset_index(drop=True)

# Define the needed variables
scar_post = scar_data["Postal Code"]
scar_lat = scar_data["Latitude"]
scar_lng = scar_data["Longitude"]

# Execute the function and make the needed calls
scar_venues = getNearbyVenues(names=scar_post, latitudes=scar_lat, longitudes=scar_lng)

M1B
M1C
M1E
M1G
M1H
M1J
M1K
M1L
M1M
M1N
M1P
M1R
M1S
M1T
M1V
M1W
M1X


In [14]:
# Time to see just how many venues were fetched per postal code.
scar_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
M1B,1,1,1,1,1,1
M1C,2,2,2,2,2,2
M1E,8,8,8,8,8,8
M1G,5,5,5,5,5,5
M1H,8,8,8,8,8,8
M1J,3,3,3,3,3,3
M1K,4,4,4,4,4,4
M1L,10,10,10,10,10,10
M1M,3,3,3,3,3,3


In [18]:
# All right, that's that. Time to use one-hot encoding to see which types of venues were popular in each postal code.
# Step 1: one hot encoding
scar_onehot = pd.get_dummies(scar_venues[['Venue Category']], prefix="", prefix_sep="")

# Step 2: add neighborhood column back to dataframe
scar_onehot['Neighborhood'] = scar_venues['Neighborhood'] 

# Step 3: move neighborhood column to the first column
fixed_columns = [scar_onehot.columns[-1]] + list(scar_onehot.columns[:-1])
scar_onehot = scar_onehot[fixed_columns]

# Step 4: Let's have a quick look at the preliminary results.
scar_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,...,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant,Women's Store
0,M1B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M1C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M1C,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M1E,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M1E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# Time to whittle that down to the average values per postal code.
scar_grouped = scar_onehot.groupby('Neighborhood').mean().reset_index()
scar_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,...,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant,Women's Store
0,M1B,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
1,M1C,0.000000,0.000,0.000000,0.000000,0.000000,0.5,0.000000,0.0,0.0,...,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
2,M1E,0.000000,0.000,0.000000,0.000000,0.125000,0.0,0.125000,0.0,0.0,...,0.125,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
3,M1G,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000000
4,M1H,0.000000,0.125,0.000000,0.125000,0.125000,0.0,0.000000,0.0,0.0,...,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.125000,0.000000,0.000000,0.000000
5,M1J,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.333333
6,M1K,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
7,M1L,0.000000,0.000,0.000000,0.200000,0.000000,0.0,0.000000,0.2,0.1,...,0.000,0.000000,0.000000,0.000000,0.000000,0.1,0.000000,0.000000,0.000000,0.000000
8,M1M,0.333333,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000,0.000000,0.000000,0.333333,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
9,M1N,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000,0.000000,0.000000,0.250000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


In [20]:
# That's quite the data set! Let's see what the most popular venue types are.
# Step 1: Define a function that will return the most popular venues per postal code
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
# Step 2: Define needed variables
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# Step 3: create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# Step 4: create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scar_grouped['Neighborhood']

for ind in np.arange(scar_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scar_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,Women's Store,College Stadium,Hobby Shop,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store
1,M1C,Bar,Construction & Landscaping,Women's Store,College Stadium,Hobby Shop,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
2,M1E,Electronics Store,Mexican Restaurant,Intersection,Bank,Medical Center,Breakfast Spot,Restaurant,Rental Car Location,Women's Store,Department Store
3,M1G,Coffee Shop,Soccer Field,Korean BBQ Restaurant,Pharmacy,Women's Store,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
4,M1H,Athletics & Sports,Thai Restaurant,Hakka Restaurant,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Women's Store,Electronics Store


In [28]:
# We've finally got the data we need, time to get clustering.
# Step 1: set number of clusters
kclusters = 3

scar_grouped_clustering = scar_grouped.drop('Neighborhood', 1)

# Step 2: run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scar_grouped_clustering)

# Step 3: Add the labels to our dataframe
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [29]:
# And now we start prepping to visualise
# Step 1: Merge the 2 data frames
scar_merged = scar_data.join(neighborhoods_venues_sorted, lsuffix="Postalcode", rsuffix="Neighborhood")
scar_merged.head()

,Postal Code,Latitude,Longitude,Borough,Neighbourhood,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge",0.0,M1B,Fast Food Restaurant,Women's Store,College Stadium,Hobby Shop,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek",2.0,M1C,Bar,Construction & Landscaping,Women's Store,College Stadium,Hobby Shop,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill",1.0,M1E,Electronics Store,Mexican Restaurant,Intersection,Bank,Medical Center,Breakfast Spot,Restaurant,Rental Car Location,Women's Store,Department Store
3,M1G,43.770992,-79.216917,Scarborough,Woburn,1.0,M1G,Coffee Shop,Soccer Field,Korean BBQ Restaurant,Pharmacy,Women's Store,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae,1.0,M1H,Athletics & Sports,Thai Restaurant,Hakka Restaurant,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Women's Store,Electronics Store


In [30]:
# With this method of joining, my cluster labels became floats but they have to be integers. Time to fix that and a few other problems!

# Step 2: Let's get rid of that 2nd neighborhood column real quick.
scar_merged_2 = scar_merged.drop(labels="Neighborhood", axis = 1)

# Step 3: Due to the limited amount of venues I retrieved, 1 postal code got left out. Time to remove that one.
scar_merged_final = scar_merged_2[scar_merged_2['Cluster Labels'].notna()]

# Step 4: Turn the floats into integers. This code will throw a warning. However, when you do use .loc it won't actually convert to an integer for some reason.
scar_merged_final["Cluster Labels"] = scar_merged_final["Cluster Labels"].astype(int)

# Step 5: One final look at the dataframe before we map the data.
scar_merged_final.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,Postal Code,Latitude,Longitude,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge",0,Fast Food Restaurant,Women's Store,College Stadium,Hobby Shop,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek",2,Bar,Construction & Landscaping,Women's Store,College Stadium,Hobby Shop,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill",1,Electronics Store,Mexican Restaurant,Intersection,Bank,Medical Center,Breakfast Spot,Restaurant,Rental Car Location,Women's Store,Department Store
3,M1G,43.770992,-79.216917,Scarborough,Woburn,1,Coffee Shop,Soccer Field,Korean BBQ Restaurant,Pharmacy,Women's Store,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae,1,Athletics & Sports,Thai Restaurant,Hakka Restaurant,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Women's Store,Electronics Store


In [32]:
# Time to create the map we need
# Step 1: Provide the location data to have the map zoom in on the correct neighbourhood
scar_latitude = 43.777702
scar_longitude = -79.233238

# Step 2: create map
map_clusters = folium.Map(location=[scar_latitude, scar_longitude], zoom_start=11)

# Step 3: set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Step 4: add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scar_merged_final['Latitude'], scar_merged_final['Longitude'], scar_merged_final['Postal Code'], scar_merged_final['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters